# Data Preparation
Prepare data from GIS output for optimization model. Requires data by PPIC region (Kern County, Southeast, or Southwest are within analysis scope).

### Inputs:
* Shapefiles of ag fields with crop type ('Crop2014'), area in acres ('Acres'), field identifier ('FID_*County*'). Expected in `../spatial/` with County name ('Fresno', 'Tulare', 'Kings', 'Kern') appended to `'_fields'` (e.g, `'Fresno_fields.shp`').
* PPIC region shapefile with region code ('PPIC_Region': 'KR', 'NE', 'NW', 'SE', 'SW')
* csv of conservation factor scores ('FID_*County*', 'TNC', 'Krat', 'Kitfox', 'Corridors', 'CPAD'). Expected in `../data/` with County name ('Fresno', 'Tulare', 'Kings', 'Kern') appended to `'_score'` (e.g, `'Fresno_score.csv'`).

### Output:
* csv of per field ag data and conservation factor scores ready for input to business as usual and strategic models.

### Process Overview:
1. Spatial join the county ag field shapefiles with PPIC region shapefile
2. For each region in the PPIC regions ('KR', 'SW', 'SE'):
    1. Lookup PPIC crop code from crop_x table
    2. Calculate price per acre
    3. Normalize conservation factors not already on a 1 - 100 scale
    4. Calculate area-weighted conservation production per field

## Inputs

In [3]:
# Inputs
counties = ['Fresno', 'Tulare', 'Kings', 'Kern']
regions = ['KR', 'SE', 'SW']
ppic = r'../spatial/PPIC_Region.shp'

# Path to output, use as input to future notebooks
out_df = f'../outputs/df_{region_code}.csv'

## Processing

In [2]:
# Import statements
import pandas as pd
import numpy as np
import geopandas as gd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
sns.set_style("white")

In [5]:
plt.rcParams["figure.figsize"] = (12,5)

### Read in Conservation Factor Data

In [31]:
field_dfs = []
for county in counties:
    df = pd.read_csv('../data/' + county + '_score.csv')
    df.rename(columns={f'FID_{county}_fields'): 'FID'}, inplace=True)
    df['County'] = county
    field_dfs.append(df)
df = pd.concat(field_dfs, sort=False)
df.head()

,FID,TNC,Krat,Kitfox,Corridors,CPAD,County
0,0,0.0,0.0,0.0,0.0,0.0,Fresno
1,1,0.0,0.0,0.0,0.0,0.0,Fresno
2,2,0.0,0.0,0.0,0.0,0.0,Fresno
3,3,0.0,0.0,0.0,0.0,0.0,Fresno
4,4,0.0,0.0,0.0,0.0,0.0,Fresno


## Read in Field Spatial Data and Join by FID

In [ ]:
#For each county:
    #Read in shapefile
    #Subset df by County
    #Join by FID
#Recombine all county data

In [38]:
shp = gd.read_file('../spatial/Kings_fields.shp')
shp.head()

,Crop2014,Acres,DWR_Standa,POLY_AREA,Shape_Leng,Shape_Area,FID_Kings,geometry
0,Cotton,37.905306,F | FIELD CROPS,37.934003,1544.750644,153397.330174,1,POLYGON Z ((17653.6521999985 -186841.011600000...
1,Cotton,53.209292,F | FIELD CROPS,53.240802,1874.607999,215330.366964,2,POLYGON Z ((32550.32499999925 -212354.40660000...
2,Cotton,29.410551,F | FIELD CROPS,29.425382,1434.154426,119020.276913,3,POLYGON Z ((39375.33980000019 -190436.41059999...
3,"Corn, Sorghum and Sudan",63.292599,F | FIELD CROPS,63.328119,2087.721625,256136.060750,4,POLYGON Z ((34595.37530000135 -194641.17689999...
4,Cotton,25.552167,F | FIELD CROPS,25.566065,1873.868169,103405.953132,5,"POLYGON Z ((35944.38980000094 -190046.9024 0, ..."


In [ ]:
shp_list = []
for county in counties:
    shp = gd.read_file(f'../spatial/{county}_fields.shp')
    shp.rename()
shp_list = [gd.read_file(f'../spatial/{county}_fields.shp') for county in counties]
fields = gd.GeoDataFrame(pd.concat(shp_list, ignore_index=True, sort=False))

In [36]:
fields.head()

,Crop2014,Acres,DWR_Standa,Shape_Leng,Shape_Area,POLY_AREA,geometry,FID_Kings,FID_Tulare,County,...,Shape_STAr,Shape_STLe,FID_1,CropPrice,WaterConsu,AcrePrice,CENTROID_X,CENTROID_Y,CENTROID_Z,CENTROID_M
0,Grapes,43.474109,V | VINEYARD,1805.885245,175933.475407,43.514680,POLYGON Z ((2657.36630000174 -137932.317900000...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Grapes,93.723265,V | VINEYARD,2574.413817,379284.598392,93.789355,POLYGON Z ((21553.0260999985 -164890.679300000...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Peaches/Nectarines,6.409884,D | DECIDUOUS FRUITS AND NUTS,721.682337,25939.880099,6.412747,POLYGON Z ((43455.04930000007 -159635.92320000...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Grapes,17.461519,V | VINEYARD,1400.484789,70664.260080,17.473741,POLYGON Z ((21610.41360000148 -159241.50689999...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Grapes,9.365854,V | VINEYARD,996.391546,37902.267829,9.373125,POLYGON Z ((15459.16169999912 -157461.65139999...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Compare inputs to PPIC report
|Region|Land <br>(1,000s acres) <br>|Applied Water<br>(1,000s of acre-ft)<br>|Revenue <br>(2010 $, millions)<br>|
|:----:|:-------------:|:-----------------:|:----------------:|
|KR    |827           |2,958              |3,948|
|SE    |1,134         |3,662              |5,930|
|SW    |1,112         |3,177              |3,917|

In [7]:
total_revenue = df['FieldCropPrice'].sum()
print (f'${total_revenue:,.2f} in annual revenue')

$7,298,194,072.79 in annual revenue


In [8]:
cropland = df['Area'].sum()
print(f'{cropland:,.0f} acres of cropland')

856,945 acres of cropland


In [9]:
applied_water = df['SavedWater'].sum()
applied_water_s = (f'{applied_water:,.0f} acre-feet of applied water')

This example {{applied_water_s}} should work

In [10]:
pd.DataFrame({'Region': [region_code], 'Land':[f'{cropland:,.0f}'], 'Water':[f'{applied_water:,.0f}'], 'Revenue':[f'${total_revenue:,.0f}']})

,Region,Land,Water,Revenue
0,KR,"856,945","2,844,471","$7,298,194,073"


In [11]:
crop_x = pd.read_csv('../data/crop_x.csv')
crop_x.head()

,Crop,Crop_PPIC
0,AlfalfaandAlfalfaMixtures,alfalfa-pasture
1,Almonds,trees-vines
2,Apples,veg-fruits
3,Avocados,trees-vines
4,Beans(Dry),veg-fruits


### Set Up Analysis
#### Cross-walk crop type to PPIC crop type

In [12]:
df = df.join(crop_x.set_index('Crop'), on='Crop')

#### Calculate PricePerAcre column
Will be used to sort crop types within each PPIC crop type to select lowest revenue crops first when fallowing fields

In [13]:
df['PricePerAcre'] = round(df['FieldCropPrice'] / df['Area'],2)

#### Normalize SavedWater, Recharge, and Depth2Groundwater

In [14]:
norm_columns = ['SavedWater', 'Recharge', 'Depth2GroundWater']
for column in norm_columns:
    new_col = str(column + '_Norm')
    df[new_col] = df[column] / df[column].max() * 100
df.head()

,Area,Crop,FieldCropPrice,SavedWater,Recharge,Kitfox,TiptonKRat,GiantGarterSnake,TNC,HabCorridors,...,ImpBirdAreas,Nitrate,ModSAGBI,Prox2Wetlands,Depth2GroundWater,Crop_PPIC,PricePerAcre,SavedWater_Norm,Recharge_Norm,Depth2GroundWater_Norm
FID,,,,,,,,,,,,,,,,,,,,,
45,70.791382,AlfalfaandAlfalfaMixtures,78836.822440,359.620220,12283.604020,100.0,100.0,0.0,94.0,0.0,...,0.0,0.000000,42.0,0.0,20.0,alfalfa-pasture,1113.65,17.043398,15.554193,100.0
72,81.343317,AlfalfaandAlfalfaMixtures,90587.985160,413.224051,1975.754272,100.0,100.0,100.0,0.0,0.0,...,0.0,100.000000,0.0,100.0,5.0,alfalfa-pasture,1113.65,19.583832,2.501812,25.0
91,28.286742,AlfalfaandAlfalfaMixtures,31501.529700,143.696647,3311.302980,100.0,100.0,0.0,0.0,0.0,...,91.0,93.333333,98.0,0.0,20.0,alfalfa-pasture,1113.65,6.810182,4.192959,100.0
116,6.073612,AlfalfaandAlfalfaMixtures,6763.878538,30.853951,417.364790,100.0,100.0,0.0,0.0,0.0,...,0.0,83.333333,75.5,0.0,11.5,alfalfa-pasture,1113.65,1.462254,0.528491,57.5
143,97.932855,AlfalfaandAlfalfaMixtures,109062.924200,497.498904,11437.290070,100.0,100.0,4.0,0.0,83.0,...,0.0,0.000000,97.0,100.0,20.0,alfalfa-pasture,1113.65,23.577851,14.482542,100.0


#### Calculate area-weighted ecosystem service values

In [15]:
df.columns

Index(['Area', 'Crop', 'FieldCropPrice', 'SavedWater', 'Recharge', 'Kitfox',
       'TiptonKRat', 'GiantGarterSnake', 'TNC', 'HabCorridors',
       'ConservationAreas', 'ImpBirdAreas', 'Nitrate', 'ModSAGBI',
       'Prox2Wetlands', 'Depth2GroundWater', 'Crop_PPIC', 'PricePerAcre',
       'SavedWater_Norm', 'Recharge_Norm', 'Depth2GroundWater_Norm'],
      dtype='object')

In [16]:
# eco_service_columns = [str(c) for c in df.columns[6:16]] + [str(d) for d in df.columns[19:]]
eco_service_columns = np.append(df.columns.values[5:16], df.columns.values[19:])
eco_service_columns

array(['Kitfox', 'TiptonKRat', 'GiantGarterSnake', 'TNC', 'HabCorridors',
       'ConservationAreas', 'ImpBirdAreas', 'Nitrate', 'ModSAGBI',
       'Prox2Wetlands', 'Depth2GroundWater', 'Recharge_Norm',
       'Depth2GroundWater_Norm'], dtype=object)

In [17]:
for eco_service in np.append(eco_service_columns[:-3], [eco_service_columns[-1]]): # exclude Recharge and Saved Water
    new_col = str(eco_service + '_Total')
    df[new_col] = df[eco_service] / 100 * df['Area']

In [18]:
df.head()

,Area,Crop,FieldCropPrice,SavedWater,Recharge,Kitfox,TiptonKRat,GiantGarterSnake,TNC,HabCorridors,...,TiptonKRat_Total,GiantGarterSnake_Total,TNC_Total,HabCorridors_Total,ConservationAreas_Total,ImpBirdAreas_Total,Nitrate_Total,ModSAGBI_Total,Prox2Wetlands_Total,Depth2GroundWater_Norm_Total
FID,,,,,,,,,,,,,,,,,,,,,
45,70.791382,AlfalfaandAlfalfaMixtures,78836.822440,359.620220,12283.604020,100.0,100.0,0.0,94.0,0.0,...,70.791382,0.000000,66.543899,0.00000,70.791382,0.000000,0.000000,29.732380,0.000000,70.791382
72,81.343317,AlfalfaandAlfalfaMixtures,90587.985160,413.224051,1975.754272,100.0,100.0,100.0,0.0,0.0,...,81.343317,81.343317,0.000000,0.00000,81.343317,0.000000,81.343317,0.000000,81.343317,20.335829
91,28.286742,AlfalfaandAlfalfaMixtures,31501.529700,143.696647,3311.302980,100.0,100.0,0.0,0.0,0.0,...,28.286742,0.000000,0.000000,0.00000,13.011901,25.740935,26.400959,27.721007,0.000000,28.286742
116,6.073612,AlfalfaandAlfalfaMixtures,6763.878538,30.853951,417.364790,100.0,100.0,0.0,0.0,0.0,...,6.073612,0.000000,0.000000,0.00000,6.073612,0.000000,5.061344,4.585577,0.000000,3.492327
143,97.932855,AlfalfaandAlfalfaMixtures,109062.924200,497.498904,11437.290070,100.0,100.0,4.0,0.0,83.0,...,97.932855,3.917314,0.000000,81.28427,97.932855,0.000000,0.000000,94.994870,97.932855,97.932855


In [19]:
df.to_csv(out_df)